In [ ]:
# A rule-based sentiment analysis

In [434]:
import plotly.express as px #visualization
import pandas as pd
from nltk import word_tokenize
import preprocessor as p
from nltk.corpus import stopwords
import numpy as np
import re
import nltk
import string
from evds import evdsAPI
import preprocessor as p
from snowballstemmer import TurkishStemmer
evds = evdsAPI('api-key')
pd.set_option('display.max_colwidth', -1)

In [435]:
df=pd.read_json('bist100twits.json')
df['text1']=[p.clean(e) for e in df['text']]
df=df[['text1',  'timestamp']]
df['date']=df['timestamp'].dt.date
df['text']=df['text1'].str.lower()

In [436]:
df=df[['text1',  'timestamp']]
df['date']=df['timestamp'].dt.date
df['text']=df['text1'].str.lower()

In [438]:
neg=pd.read_csv('negative_words_tr.txt')
pos=pd.read_csv('positive_words_tr.txt')
PUNCT_TO_REMOVE = string.punctuation
stoplist=stopwords.words('turkish')
def remove_punctuation(text):# notktalama işaretleri
    """custom function to remove the punctuation"""
    return text.translate(str.maketrans('', '', PUNCT_TO_REMOVE))

df['text'] = df['text'].apply(lambda text: remove_punctuation(text))
def remove_stopwords(text):
    """custom function to remove the stopwords"""
    return " ".join([word for word in str(text).split() if word not in stoplist])

df['text_stop'] = df['text'].apply(lambda text: remove_stopwords(text))

In [439]:
def stemming_tokenizer(text): 
    stemmer = TurkishStemmer()
    return [stemmer.stemWord(w) for w in word_tokenize(text)]
df['text_token'] = df['text_stop'].apply(lambda text: stemming_tokenizer(text))

In [440]:
def word_finder1(x):
    df_words = set(" ".join(x).split(" "))
    extract_words =  set(neg.eski.to_list()).intersection(df_words)
    return ', '.join(extract_words)
def word_finder2(x):
    df_words = set(" ".join(x).split(" "))
    extract_words =  set(pos.olan.to_list()).intersection(df_words)
    return ', '.join(extract_words)
df['neg'] = df.text_token.apply(word_finder1)
df['pos'] = df.text_token.apply(word_finder2)

In [441]:
df[['text_token', 'neg', 'pos']].sample(3)

,text_token,neg,pos
9608,"[i̇nsan, sağlık, zenginlik, dilemeyinşans, diley, titanik, zengi, sağlıklı, idi, şansli, olan, kurtul]",,"sağlıklı, zenginlik"
3919,"[bist, elliot, wawe, plea]",,
3869,"[71, aç, 8, tekrar, söyülüyor, endeks, stratej, 25, üzer, kalıç, pozisyon, açar, alt, oluşacak, hareket, temkinli, olur, akşa, youtube, kanal, ilgil, güncel, video, paylaşacak]","alt, kanal",güncel


In [442]:
df['len_pos'] = df.pos.apply(lambda x: len(x.split(', ')))
df['len_neg'] = df.neg.apply(lambda x: len(x.split(', ')))

In [443]:
df['len_neg']=df['len_neg']-1
df['len_pos']=df['len_pos']-1

In [444]:
df['text_count'] = df['text_stop'].str.split().apply(len).value_counts()
df['polarity'] = df['len_pos'] - df['len_neg']
df['sent_rate']=(df['len_pos'] + df['len_neg'])/ df['text_count']

In [445]:
d=pd.DataFrame(df.groupby('date')['polarity'].mean())
d=d.reset_index()
d['date']=pd.to_datetime(d['date'])
d=d.sort_values(by='date')
fig = px.line(d, x=d.date, y=d.polarity, title='polarite')
fig.show()

In [446]:
bist=evds.get_data(['TP.MK.F.BILESIK'] , startdate="01-03-2020", enddate='01-05-2020')
bist.columns=['tarih', 'kapanış']
fig = px.line(bist, x=bist.tarih, y=bist.kapanış, title='BİST 100 Endeksi 1 Mart - 1 Mayıs')
fig.show()

# kaynaklar: 
* https://stackoverflow.com/questions/51413271/pandas-loop-through-list-and-find-word-from-list-in-column-create-new-colum
* https://monkeylearn.com/sentiment-analysis/
 * tweet alma komutu: twitterscraper Bist100  -l 10017-01-01 -ed 2017-06-01 -o tweets.json
 * https://pypi.org/project/tweet-preprocessor/
 * https://pypi.org/project/evds/
 * https://github.com/taspinar/twitterscraper
 * https://github.com/otuncelli/turkish-stemmer-python
 * https://www.kaggle.com/rtatman/sentiment-lexicons-for-81-languages
 * https://towardsdatascience.com/extracting-twitter-data-pre-processing-and-sentiment-analysis-using-python-3-0-7192bd8b47cf